https://github.com/lmoroney/dlaicourse/blob/master/Course%201%20-%20Part%208%20-%20Lesson%202%20-%20Notebook.ipynb

In [38]:
#import sys
#!{sys.executable} -m pip install --upgrade pip
#!{sys.executable} -m pip install --upgrade tensorflow
#import tensorflow
#print(tensorflow.__version__)

In [39]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip
    
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
zip_ref.close()

# Directory with our training horse pictures
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('/tmp/horse-or-human/humans')


--2020-05-20 11:03:22--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c09::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M   188MB/s    in 0.8s    

2020-05-20 11:03:23 (188 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [40]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [41]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/horse-or-human/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 1027 images belonging to 2 classes.


In [42]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['AUC'])

history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=5,
      verbose=1)

Epoch 1/5
8/8 [==============================] - 25s 3s/step - auc: 0.5120 - loss: 6.6686
Epoch 2/5
8/8 [==============================] - 21s 3s/step - auc: 0.7313 - loss: 0.7927
Epoch 3/5
8/8 [==============================] - 21s 3s/step - auc: 0.9440 - loss: 0.3117
Epoch 4/5
8/8 [==============================] - 25s 3s/step - auc: 0.9621 - loss: 0.2674
Epoch 5/5
8/8 [==============================] - 21s 3s/step - auc: 0.8747 - loss: 0.6909


In [43]:
# AUC calculated with model.evaluate

model.evaluate(train_generator)

9/9 [==============================] - 9s 1s/step - auc: 0.9987 - loss: 0.1195


[0.11946665495634079, 0.9986773729324341]

In [44]:
import numpy as np

In [45]:
# Predictions

x = model.predict(train_generator)
x = np.asarray([i[0] for i in x])
x

array([0.0302211 , 0.99999934, 0.07031509, ..., 0.89338297, 0.06161412,
       0.9366726 ], dtype=float32)

In [46]:
# AUC calculated with roc_auc_score based on predictions

from sklearn import metrics
metrics.roc_auc_score(train_generator.labels, x)

0.5273529411764706

In [47]:
# Suggestion from https://stackoverflow.com/a/61899838/5203413

print("Labels shape: ")
print(train_generator.labels.shape)

print("Predictions shape: ")
print(x.shape)

print("First 10 and last 10 values of labels:")
print(train_generator.labels[:10])
print(train_generator.labels[-10:])

print("First 10 and last 10 values of predictions:")
print(x[:10])
print(x[-10:])

m = tf.keras.metrics.AUC()

m.update_state(train_generator.labels, x) # assuming both have shape (N,)

r = m.result().numpy()

print("AUC calculated with keras.metrics on predictions:")
print(r)

Labels shape: 
(1027,)
Predictions shape: 
(1027,)
First 10 and last 10 values of labels:
[0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1]
First 10 and last 10 values of predictions:
[0.0302211  0.99999934 0.07031509 0.9291356  0.9579649  0.904967
 0.8771623  0.9910754  0.98487496 0.9999261 ]
[0.96782637 0.38549215 0.98972666 0.99019325 0.10420808 0.00382492
 0.97860146 0.89338297 0.06161412 0.9366726 ]
AUC calculated with keras.metrics on predictions:
0.52777994
